# 6장. 생성
어느 분류에 속하는 데이터 수가 다른 분류에 속하는 데이터에 비해 매우 적은 상태일 경우 데이터를 생성해야 함.

> #### 불균형한 데이터를 대처하는 방법
* 머신러닝 모델을 만들 때 가중치를 부여하는 법.
* 데이터를 변경하여 불균형한 상태 해소.
    * 적은 데이터를 늘리는 오버샘플링 
    * 많으느 데이터를 줄이는 언더샘플링  
    * 두 가지 방법을 모두 사용하는 방법 

=> 가중치를 부여하는 방법은 머신러닝 모델의 종류에 따라 특성이 달라지고 사용하기 까다로우며, 제공되는 라이브러리에 따라서는 이용할 수 없는 경우도 있다.

    
    
    

## 6.1 언더샘플링으로 데이터 불균형 조정하기
> 1) 데이터 선택 방법은 중복선택 (같은 데이터를 두 번 이상 선택)을 허용하지 않는 것이 좋다.
* 중복 선택으로 특정 데이터가 여러 번 선택되면 데이터가 편중될 수 있어 과학습이 발생되기 쉬우므로

> 2) 샘플링 수가 적을 때 데이터 편향이 없는 랜덤 샘플링을 구현하기 위해 <br>
미리 데이터를 클러스터링 하여 작성된 크러스터별로 샘플링을 실행하는 방법 있음.
* 그러나 언더샘플링은 데이터를 걸러서 정보량을 적게 만드는 방법이기 때문에 되도록 사용하지 않는 게 좋음.

> 3) 오버샘플링과 언더샘플링을 같이 사용하는 법
* 오버샘플링으로 악영향이 미치지 않는 정도까지만 데이터양이 적은 쪽 데이터를 늘리고,
* 언더샘플링으로 불균형이 해소되는 정도까지만 데이터양이 많은 쪽 데이터를 줄이는 것.

## 6.2 오버샘플링으로 데이터 불균형 조정하기

#### 랜덤샘플링으로 오버샘플링 할 경우 문제
* 간단하지만, 완전히 똑같은 데이터가 생성되기 때문에 과학습이 발생할 수 있음.

> #### SMOTE (Synthetic Minority Over-samplling Technique) 기법 
* SMOTE는 오버샘플링을 할 때 원본 데이터를 기반으로 새로운 데이터를 생성하는 방법.
* SMOTE의 데이터 생성부분 알고리즘 개요
    * 대상선택 -> 대상 근처의 데이터를 선택 후보로 지정 -> 선택 후보 중 하나의 데이터를 선택 -> 두 가지 데이터로 새로운 데이터 생성
* SMOTE로 생성된 데이터는 원본 데이터와 같은 특성을 유지하면서 약간의 노이즈를 더한 데이터를 의미한다. 
    * 그 결과 단순 랜덤 샘플링으로 원본 데이터를 복사하는 것 보다 좀 더 자연스럽게 발생한 데이터에 가까운 값을 가짐.

* SMOTE기법 단점
    * SMOTE는 직선상값을 샘플링하는 방법이기 때문에, 차원 수 (생성할 데이터의 열수)가 큰 경우는 편중된 샘플링이 되기 쉽다.
         * 차원 수가 큰 경우엔 언더샘플링과 배깅을 조합하여 예측 모델을 구축하는 게 좋음.

### Q1. 오버샘플링
장애가 발생하지 않은 레코드가 927건, 장애가 발생한 레코드가 73건 이므로 , <br>
장애가 발생한 레코드를 SMOTE로 오버샘플링하여 장애가 발생하지 않은 레코드의 건수에 가깝게 만든다. <br>
SMOTE의 k매개변수는 5로 지정. 


In [1]:
import pandas as pd
import numpy as np

In [2]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
production_tb=pd.read_csv('./data/production.csv',encoding='UTF-8')
production_tb.head()

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [27]:
from imblearn.over_sampling import SMOTE

# ratio는 불균형 데이터에서 적은 열의 데이터를 많은 열의 몇 할까지 늘릴지 설정한다.

# auto는 수가 같아질 때 까지 늘리고, 0.5로 설정하면 5할까지 데이터를 늘린다.

# k_neighbors는 smote의 k매개변수

sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=71)

# 오버샘플링을 실행한다.
balance_data, balance_target = sm.fit_sample(production_tb[['length','thickness']], production_tb['fault_flg'])


* 이 예시에서 이용하는 라이브러리에는 범줏값을 오버샘플링 할 수 없다. 
    * 범줏값을 오버샘플링 하려면 라이브러리의 내용을 변경하거나 SMOTE함수를 처음부터 만들어야 함.

In [28]:
len(production_tb)

1000

In [29]:
production_tb.groupby(['fault_flg']).count()

,type,length,thickness
fault_flg,,,
False,948,948,948
True,52,52,52


In [48]:
len(balance_data)

1896

### 결과 확인
원본 테이블에서 False는 948개, True는 52개 였는데 오버샘플링 후 True도 948개로 False데이터 개수와 같아짐 .

In [46]:
# Series타입 카운트 할때는 value_counts
balance_target.value_counts()

True     948
False    948
Name: fault_flg, dtype: int64